<img src="pr1.png" />

In [10]:
import cvxpy
import gurobipy
from cvxpy import *
import numpy as np

In [18]:
np.random.seed(42)

# initial matrix
m = np.array([[ 5 , 5 , 10, 10, 10 ],
              [ 5 , 5 , 10, 20, 10 ],
              [ 0 , 5 , 5 , 10,  5 ],
              [ 0 , 0 , 0 , 5 ,  0 ]])

N, M = m.shape
C = N * M

# delta matrix
d = np.ones((N, M)) * 6
d = d - m

# cost to transfer the ground from the (i1, j1) to (i2, j2)
def cost(i1, j1, i2, j2):
    assert(i1 < N and i2 < N and j1 < M and j2 < M and 
           i1 >= 0 and i2 >= 0 and j1 >= 0 and j2 >= 0)
    
    if i1 == i2 and j1 == j2:
        return 1000000
    
    return np.sqrt(np.power(np.abs(i1 - i2), 2) + np.power(np.abs(j1 - j2), 2))

m = m.reshape(C,1)
d = d.reshape(C,1)

f = Int(C * C)
c = np.zeros(C * C)

# fill the cost matrix
for i1 in xrange(N):
    for j1 in xrange(M):
        for i2 in xrange(N):
            for j2 in xrange(M):
                c[(i1 * M + j1) * C + (i2 * M + j2)] = cost(i1, j1, i2, j2)
                
# Each cell can either be only a consumer or a "donor" in my model.
# This simplification just introduces a new cutting plane which doesn't
# affect the global optimum (I had the same result without "donors").
# This restriction gave me more integers when I was trying to solve with 
# the general LP solver.
def pred(x):
    if x < 0:
        return 1
    
    return 0

pd = np.vectorize(pred)
is_donor = pd(d).reshape(C,)

flow_network_constraints = [ (-is_donor[i] * cvxpy.sum_entries(f[i*C:i*C + C]) + \
                             (1 - is_donor[i]) * cvxpy.sum_entries(f[i:C*C:C]) == d[i]) \
                            for i in xrange(C) ]
fnc_not_donors = [ (1 - is_donor[i]) * cvxpy.sum_entries(f[i*C:i*C + C]) == 0 for i in xrange(C) ]
fnc_donors = [ is_donor[i] * cvxpy.sum_entries(f[i:C*C:C]) == 0 for i in xrange(C) ]

flows_are_positive = [ (f[i] >= 0) for i in xrange(C * C) ]

obj = Minimize(c * f)

constraints = flows_are_positive + flow_network_constraints + fnc_not_donors + fnc_donors

#print installed_solvers()

prob = Problem(obj, constraints)
res = prob.solve(feastol=1e-8) # verbose=True, 
print "Result: ", res
eps = 0.0000000001
print "Flows size: ", f.value[np.where( f.value > eps )] 
# print cvxpy.sum_entries(f[13:C*C:C]).value

Result:  95.4910638371
Flows size:  [[ 1.  1.  1.  1.  4.  1.  3.  2.  1.  1.  1.  5.  6.  2.  1.  3.  4.]]


I've tried several similar min cost max flow models. All of them gave the same result as this one that is constrained to have an integer valued flows.

<img src="pr2.png" />

The LP formulation in terms of MCMF here will be as follows:

* we will think of a bipartite graph where the left side $L$ will be the client nodes (sources), and the right side $R$ - all the facilities nodes.

* edge $e = (u, v), u \in L, v \in R$ will have the cost of assigning the $u$'th client to $v$'th facility.
* each $u \in L$ node will generate flow of size $1$.
* $s$ will be a sink node. It's $b_s$ value will be $-\Vert C\Vert$ (each client must finally be attached to a facility). Each $v \in R$ will have an edge to $s$ with a capacity eqauls to the $v$'s facility capacity.


<img src="img.png" height="150" width="300" />

This LP formulation doesn't take the facility oppening cost into account. But it can be considered as a lower bound. In the following B-n-B implementation I will be making a branch on every

In [87]:
# initialization

import numpy as np
import scipy as sp
from numpy import random
from numpy import matlib
import matplotlib.pyplot as plt
from cvxpy import *
import cvxpy

%matplotlib inline

np.random.seed(1)

C = 40; # number of clients
clients = np.random.rand(2,C); #client positions
F = 20; #number of facilities
facilities = np.random.rand(2,F);

#capacities = np.ones((F,))*4; #maximum number of clients per facility
capacities = np.array([2, 3, 4, 2, 5, 3, 2, 1, 2, 3, 4, 3, 2, 1, 2, 2, 2, 3, 2, 3]);

dx = np.matlib.repmat(clients[0,:],F,1) - np.matlib.repmat(facilities[0,:],C,1).transpose();
dy = np.matlib.repmat(clients[1,:],F,1) - np.matlib.repmat(facilities[1,:],C,1).transpose();

assignment_costs = 3*(dx*dx + dy*dy); #the assignment cost is the distance squared

#opening_costs = np.ones((F,));
opening_costs = np.array([1, 2, 1, 2, 1, 1.2, 1.3, 2, 1, 1, 1.4, 2, 1, 1, 1, 1, 1.5, 1.3, 2, 1]);


def plot_solution(facilities_opened, clients_assignment):
    plt.scatter(clients[0,:], clients[1,:], s=50.0, c=clients_assignment)
    plt.scatter(facilities[0,:], facilities[1,:], s=200.0, c=range(F), linewidth = [5*el for el in facilities_opened])
    plt.show()

In [89]:
POS_INF = 1000000L

# solves the problem for the specific node.
# node is identified by the set of facilities we fix as closed/opened.
def lp_solve(closed, opened):
    global F, C, nodes_visited, clients, facilities, capacities, assignment_costs, opening_costs
    
    # check if there's enough total capacity of the facilities
    if np.sum(capacities) - np.sum(capacities[closed.keys()]) < C: 
        return [POS_INF, []]
    
    def cost(cl, fc):
        if fc in closed:
            return POS_INF

        return assignment_costs[fc, cl]

    f_1 = Int(C, F)
    f_2 = Int(F) # edges directing to sink
    c = np.zeros((C, F))

    for i in xrange(C):
        for j in xrange(F):
            c[i, j] = cost(i, j)

    constraints = [ (cvxpy.sum_entries(f_1[i,:]) == 1) for i in xrange(C) ] + \
                  [ (cvxpy.sum_entries(f_1[:,i]) - f_2[i] == 0) for i in xrange(F) ] + \
                  [ f_1[i, j] >= 0 for i in xrange(C) for j in xrange(F) ] + \
                  [ f_2[i] >= 0 for i in xrange(F) ] + \
                  [ f_2[i] <= capacities[i] for i in xrange(F) ] + \
                  [ cvxpy.sum_entries(f_2) == C ]
    
    obj = Minimize(cvxpy.sum_entries(vec(f_1).T * vec(c)))
    prob = Problem(obj, constraints)
    res = prob.solve()
    eps = 1e-10
    
    opened = np.where(f_2.value > eps)[0]
    res += np.sum(opening_costs[opened])
    return [res, opened]

def branch_and_bound(iterations = 1):
    incumbent_value = POS_INF
    incumbent = {}
    
    queue = [{}]
    
    while queue and iterations > 0:
        iterations -= 1
        node = queue.pop(0)
        res, opened = lp_solve(node)

        if res < incumbent_value:
            incumbent_value = res
            incumbent = node
            
            for i in opened:
                nc = node.copy()
                nc[i] = True
                queue.append(nc)
        
        print "Node: ", node, "; res: ", res
                
    print incumbent_value, incumbent
        
branch_and_bound(10000)

Node:  {} ; res:  29.2803830784
Node:  {0: True} ; res:  28.7338806895
Node:  {1: True} ; res:  27.7995889499
Node:  {2: True} ; res:  28.2962736895
Node:  {3: True} ; res:  27.857471729
Node:  {4: True} ; res:  30.0300479834
Node:  {5: True} ; res:  28.3325779364
Node:  {6: True} ; res:  28.4388367557
Node:  {7: True} ; res:  27.280346103
Node:  {8: True} ; res:  28.3739595842
Node:  {9: True} ; res:  28.2803453239
Node:  {10: True} ; res:  30.1377677306
Node:  {11: True} ; res:  28.4477913065
Node:  {12: True} ; res:  28.3542398606
Node:  {13: True} ; res:  28.2940981636
Node:  {14: True} ; res:  28.7608905096
Node:  {15: True} ; res:  29.057635107
Node:  {16: True} ; res:  27.7803893712
Node:  {17: True} ; res:  27.9919159975
Node:  {18: True} ; res:  28.2428476228
Node:  {19: True} ; res:  29.505118355
Node:  {0: True, 1: True} ; res:  27.3862980285
Node:  {0: True, 2: True} ; res:  27.7497368367
Node:  {0: True, 3: True} ; res:  27.6626134552
Node:  {0: True, 4: True} ; res:  29.7

Previous solution via simulated annealing gave me $22.89$ last time. Now it's $22.41$. And GUROBI gives $\approx 21.8$

<img src="pr3.png" />